In [1]:
import requests
from bs4 import BeautifulSoup

income_statement = {}

url = "https://finance.yahoo.com/quote/AAPL/financials?p=AAPL"

headers = {"User-Agent" : "Chrome/96.0.4664.110"}
page = requests.get(url, headers=headers)
page_content = page.content
soup = BeautifulSoup(page_content,"html.parser")
tabl = soup.find_all("div" , {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
for t in tabl:
    rows = t.find_all("div" , {"class": "D(tbr) fi-row Bgc($hoverBgColor):h"})
    for row in rows:
        income_statement[row.get_text(separator="|").split("|")[0]] = row.get_text(separator="|").split("|")[1]
        


In [2]:
income_statement

{'Total Revenue': '394,328,000',
 'Cost of Revenue': '223,546,000',
 'Gross Profit': '170,782,000',
 'Operating Expense': '51,345,000',
 'Operating Income': '119,437,000',
 'Net Non Operating Interest Income Expense': '-106,000',
 'Other Income Expense': '-228,000',
 'Pretax Income': '119,103,000',
 'Tax Provision': '19,300,000',
 'Net Income Common Stockholders': '99,803,000',
 'Diluted NI Available to Com Stockholders': '99,803,000',
 'Basic EPS': '-',
 'Diluted EPS': '-',
 'Basic Average Shares': '-',
 'Diluted Average Shares': '-',
 'Total Operating Income as Reported': '119,437,000',
 'Total Expenses': '274,891,000',
 'Net Income from Continuing & Discontinued Operation': '99,803,000',
 'Normalized Income': '99,803,000',
 'Interest Income': '2,825,000',
 'Interest Expense': '2,931,000',
 'Net Interest Income': '-106,000',
 'EBIT': '122,034,000',
 'EBITDA': '133,138,000',
 'Reconciled Cost of Revenue': '223,546,000',
 'Reconciled Depreciation': '11,104,000',
 'Net Income from Conti